# About the code

This is an additional jupyter notebook used to stabilize the opo mode. Details on stabilizing the cavity is mentioned in Section S3 of the supplementary information

In [4]:
## Import all the necessary libraries (to install use pip or use directly the github of said libraries or use requirements.txt)

import pyvisa
import numpy as np
import matplotlib.pyplot as plt
import time

%matplotlib notebook     

In [6]:
#Input variables:

Data_Path=#"path_to_save_data" 

#VISA instrument names
fgen_visa = #visa name of funciton generator EX. "GPIBO::12:INSTR"
osc_visa = #visa name of oscilloscope
PID_visa = #visa name of PID
MotionControll_visa= #visa name of Rotation Angle Filter 


#Function Generator sweeping variables
# These variables set the values of the function generator in order to sweep the cavity for resonance peaks

func_symmetry = 50 # symmetry of functions (e.g 100, 50)
phase = 0 #Phase angle
volt_high = 5 # Maximum voltage of function (in volts)
volt_low = 0 #Minimum voltage of function (in volts)
freq = 100 # Frequency of function (in Hertz)
func_channel= 2  #Channel Used by the sweep in the function generator
#ADD putput inpedance
#waveform is hardcoded in the programm

# Value for the  modulation

modulation_simmetry=50
modulation_phase = 0 #Phase angle
modulation_volt_high = 0 # Maximum voltage of function (in volts)
modulation_volt_low = -2.4 #Minimum voltage of function (in volts)
modulation_freq = 10e3 # Frequency of function (in Hertz)
modulation_channel= 1 #Channel Used by the Modulation in the function generator
#ADD putput inpedance
#waveform is hardcoded in the programm

#Oscilloscope variables
# These variables set the oscilloscope display and data recording settings

opo_ch_source = 1 # stores the channel input for the opo signal (e.g 1)
error_ch_source = 4 # stores the channel input for the error signal (e.g 1)
volt_ch_source = 3 # stores the channel input for the driving voltage signal (e.g 1)
modulation_ch_source = 2 # stores the channel input for the driving modulation signal (e.g 1)
opo_volts_per_div = 0.2 # sets the vertical voltage sscale of oscilloscope opo input(in volts)
error_volts_per_div = 0.2 # sets the vertical voltage sscale of oscilloscope error signal input (in volts)
voltage_volts_per_div = 0.2 # sets the vertical voltage sscale of oscilloscope voltage signal input(in volts)
trigger_level = 0.6 # sets the trigger level on oscilloscope in order to trigger the scope and collect data (in volts)
nbr_data_points= 200000 # Number of data points in the oscillo # Doesnt work do it manually


#PID parameters
setpoint_mode = 0 # sets the set point mode, 0 for internal, 1 for external
offset = 0.3 #sets the offset of the PID box (in volts)
max_out=offset*2 #Maximum Voltage given by the PID (in volts)
min_out=0       # Minimum Voltage given by the PID (in volts)
p_gain = 0.4 #sets the proportional gain of PID (in ??)
i_gain = 14.8E3 # sets the integral gain of PID (in ??)
#Initial PID BOX variables
#These variables set the initial mode and values of the PID box
manual_output = 0.64 #The voltage output in volts of the PID box in manual mode


## Components

In [ ]:
# Define a class for a PID object that can control the PID

class PID :
   
    def __init__(self, path=PID_visa): 
        
        # path : String like pointing to the visa adress of the PID
        # Initiate an instance of the PID, use the visa adress "path"
        
        rm = pyvisa.ResourceManager()
        self.conn = rm.open_resource(path)
        self.dType = self.conn.query('*IDN?')
        print ("PID connected : " + self.dType)  #Indicate to the user that the PID is connected
            
    def set_offset(self,V=0.64):
        
        # V : Float for the voltage 
        # Set the offset of the PID to "V" (in V)
        
        self.conn.write("SNDT 1, 'AMAN 0'")
        self.conn.write("SNDT 1, 'MOUT "+str(V)+"'") 
    
    def stop(self):
        #Stop the Pid, no argument
        
        self.conn.write("SNDT 1, 'AMAN 0'")
        
    def setup_control(self,setpoint_mode=0, offset=3, p_gain=0.2, i_gain=92):
        # setpoint_mode : int defining the setpoint mode (0= internal, 1=external)
        # offset : Float defining the offset voltage (in V)
        # p_gain : Float Defining the p_gain of the PID
        # i_gain : Float Defining the i_gain of the PID
        # Setup all the PID values above before any measurements 
        
        self.conn.write("SNDT 1, 'INPT "+ str(setpoint_mode) +"'")
        self.conn.write("SNDT 1, 'OFST "+str(offset) + "'")
        self.conn.write("SNDT 1, 'OCTL 1'")
        self.conn.write("SNDT 1, 'GAIN "+str(p_gain)+"'")
        self.conn.write("SNDT 1, 'PCTl 1'")
        self.conn.write("SNDT 1, 'ICTL 1'")
        self.conn.write("SNDT 1, 'INTG "+str(i_gain)+"'")
        
    
    def control(self,int_set):
        # int_set : Float defining the internal setpoint (in V)
        # Enable the control by the PID to the setpoint "int_set" (in V)
        
        int_set=int(1000*int_set)/1000
        self.conn.write("SNDT 1, 'SETP "+ str(int_set) +"'")
        self.conn.write("SNDT 1, 'AMAN 1'")
    
    def get_out (self):
        # Get the voltage send by the PID (in V) 
        
        self.conn.write("SNDT 1, 'OMON?'")
        try:
            a=float(self.conn.query("GETN? 1, 120")[6:-3])
        except:
            a=self.get_out()
        finally:
            return(a)     

In [ ]:
# Define a class for a FunctionGenerator object that can control the FunctionGenerator

class FunctionGenerator :
    
    def __init__(self, path=fgen_visa):
        # path : String like pointing to the visa adress of the FunctionGenerator
        # Initiate an instance of the FunctionGenerator, use the visa adress "path"
        
        rm = pyvisa.ResourceManager()
        self.conn = rm.open_resource(path)
        self.dType = self.conn.query('*IDN?')
        print ("Function generator Connecetd : " +self.dType) #Indicate to the user that the Function Generator is connected
            
    
    def set_function_DC(self,V,func_channel=2):
        self.conn.write("OUTP"+str(func_channel)+":LOAD INF")
        self.conn.write("SOURce"+str(func_channel)+":FUNCtion DC")
        self.conn.write("SOURce"+str(func_channel)+":VOLTage:OFFSet "+str(V))
        
    def stop(self,func_channel=2):
        #func_channel : int indicating the channel which we want to stop
        #Stop the channel "func_channel"
        
        self.conn.write("OUTPut"+str(func_channel)+" OFF")
    
    def start(self,func_channel=2):
        #func_channel : int indicating the channel which we want to stop
        # Start the channel "func_channel"
        
        self.conn.write("OUTPut"+str(func_channel)+" ON")

## Connection

In [ ]:
# Try to create an instance of all the previously defined electric devices, no input needed.
# Use the variables m,o,p,f,r for : the MotionController, the Oscilloscope, the PID, the FunctionGenerator and the RotationStage
# Return theses instances to be used later


def Setup():
    global p,f
    p=PID() # Connect the PID 
    f=FunctionGenerator() # Connect the Function Generator               

In [ ]:
# Print which visa objects are connected to the computer
# Usefull to check which device is connected in case of missfunction

def WhatConnected():
    rm=pyvisa.ResourceManager() #Acces to connected visa
    print(rm.list_resources())

## Setpoint Control

The parasitic green signal linearly increases or decreases if the cavity length is drifting to the specific direction. We receive the PID signal and determine to which direction cavity length is drifting and compensate it by giving additional dc voltage to the piezo attached to the mirror.

In [ ]:
# volt : Float, DC voltage of the piezo
# It returns volt (Float), the new DC voltage of the piezo
# This function changes by 1% the DC voltage acoording to the PID response
# It goes in the same direction as the piezo to help him lock
# This is just "one loop" of the computer additional control loop

def Control_Setpoint(volt):
    p.conn.query("GETN? 1, 120") # Try to get rid of the paraistic information buses in the PID
    out=p.get_out() # Get the PID output
    if out-min_out>(max_out-min_out)*6/10: # If Pid is sending positive voltage
        volt=volt*101/100 # Increase DC voltage
    if out-min_out<(max_out-min_out)*4/10: # If Pid is sending negative voltage
        volt=volt*99/100 # Decrease DC voltage
    f.set_function_DC(volt,func_channel=func_channel) # Execute the DC voltage change
    return(volt)

In [ ]:
# This is the total control loop for the PID that uses the previous function
# It stops whenever max voltage is reached or whenever user interrupt


def Stabilize_Setpoint():
    volt=float(f.conn.query("SOURce"+str(func_channel)+":VOLTage:OFFSet?"))
    volt=float(f.conn.query("SOURce"+str(func_channel)+":VOLTage:OFFSet?")) # get current DC voltage
    p.conn.query("GETN? 1, 120")
    out=p.get_out()
    time.sleep(0.1)
    try:
        while volt<10: # while volt is in good range
            time.sleep(0.1)
            volt=Control_Setpoint(volt) # we excute the control loop
    except KeyboardInterrupt: # if user interrupt we stop the loop
        pass

## Stabilize OPO

In [17]:
Setup()

PID connected : Stanford_Research_Systems,SIM900,s/n072297,ver3.5

Function generator Connecetd : Agilent Technologies,33622A,MY53800239,A.01.08-2.25-03-64-02



In [18]:
Stabilize_Setpoint()